# Greece trip mapping
### Goal of the project:
Mapping an optimized route for visiting all the points of interest I chose for the trip.\
I plan to visit Greece next year for a duration of 3 to 4 weeks. I wanted to create this little project to help me optimize the routing of the trip (choosing which city to visit next based on the distance between each city).

### Inspirations: 
Mainly this tutorial: https://towardsdatascience.com/how-to-plot-a-route-on-a-map-fb900a7f6605; that I adapted with another API and some other changes

## Installs and Imports
We install folium and routingpy only as all the other needed packages should already be installed by default with your python version.If you're missing a package, just type `pip install packagename` in a .py script or `%pip install packagename` if you're using a jupyter notebook.\
We then import all the needed packages :
- Nominatim from geopy will help us retrieve latitude and longtitude from a string address
- ORS from routingpy will allow us to contact the ORS API and get the routing
- folium will allow us to map our routing visually
- pandas will help us with managing data

In [1]:
#%pip install folium
#%pip install routingpy

In [2]:
from geopy.geocoders import Nominatim
from routingpy import ORS
import folium
import pandas as pd

## Retrieve the locations' latitude and longitude from addresses

In [3]:
def get_lat_long_from_address(address):
    """
    get_lat_long_from_address retrieves latitude and longitude of an address.

    :address: a string containing an address
    :return: returns a tuple containing the longitude then the latitude of the address param
    """ 
    locator = Nominatim(user_agent='myGeocoder')
    location = locator.geocode(address)
    return location.longitude, location.latitude

In [ ]:
import ipywidgets as widgets
from IPython.display import display
addresses = []
add_place = True
valid_place = False
output = widgets.Output()

def quit_button_click(btn_object):
    add_place = False
    
quit_button = widgets.Button(description="Quit adding places")

while add_place:
    valid_place = False
    while not valid_place:
        user_input = input("Add a place to the trip: (Press enter to validate your entry) ")
        if user_input == "":
            print("Invalid input (Cannot be empty)")
        else:
            addresses.append(user_input)
            print((user_input), "added to the list")
            valid_place = True
       
        quit_button.on_click(quit_button_click)  
        display(quit_button)
        output

Add a place to the trip: (Press enter to validate your entry) athens
athens added to the list


Button(description='Quit adding places', style=ButtonStyle())

In [ ]:
# Empty list of addresses
addresses = []
add_place = True
valid_place = False
more_places = False
    
while add_place:
    while not valid_place:
        user_input = input("Add a place to the trip: (Press enter to validate your entry) ")
        if user_input == "":
            print("Invalid input (Cannot be empty)")
        else:
            addresses.append(user_input)
            print((user_input), "added to the list")
            valid_place = True
            more_places = False
            break;

    while not more_places:
        user_input = input("Add more places? (y/n)")
        if user_input != 'n' and user_input != 'y':
            print("Invalid input, please type either 'y' or 'n'")
        else :
            if user_input == 'n':
                add_place = False
            elif user_input == 'y':
                more_places = True
                valid_place = False
            break

In [ ]:
# We then get the coordinates of the addresses in the list using a for loop and our function
coords = tuple(get_lat_long_from_address(addr) for addr in addresses)
coords2 = [(a, b) for idx, a in enumerate(coords) for b in coords[idx + 1:]]

## Get route from point A to point B
In the tutorial I followed, the API used was not free to use (X first requests were free, but you needed to put your credit card in at the beginning).\
I chose to find another API, free to use, and I found Openrouteservice (ORS) based on OpenStreetMap. \
You just need to log onto their website https://openrouteservice.org/ and create a token in your dashboard. \
It is limited to 2000 requests everyday, so not really well suited for commercial use but it is perfect for a small project like this.

In [6]:
# We read the file where we put our secret token
api_key = open("api_key.txt", "r")
api_key = api_key.read()

In [7]:
# An empty list ready to receive the routes between the addresses we defined earlier
routes = []

# We create a connection with the API
client = ORS(api_key=api_key)
# We loop over the coordinates
if len(coords2) > 1:
    for i in range(len(coords2)-1):
        for n in range(len(coords2[i])):
            lat1, lon1, lat2, lon2 = coords2[i][n][0], coords2[i][n][1], coords2[i+1][n][0], coords2[i+1][n][1]
            """
            We get the directions between an address and the next one in the list
            We define a profile, here I chose 'driving-car', but you can select 'foot-walking' for example
            Full list of path parameters: https://openrouteservice.org/dev/#/api-docs/v2/directions/{profile}/get
            """
            route = client.directions(locations=coords, profile='driving-car')
            routes.append(route)
else:
    for n in range(len(coords2)):
        lat1, lon1, lat2, lon2 = coords2[n][0], coords2[n][1], coords2[n][0], coords2[n][1]
        """
        We get the directions between an address and the next one in the list
        We define a profile, here I chose 'driving-car', but you can select 'foot-walking' for example
        Full list of path parameters: https://openrouteservice.org/dev/#/api-docs/v2/directions/{profile}/get
        """
        route = client.directions(locations=coords, profile='driving-car')
        routes.append(route)

In [8]:
def create_map(routes, coords):
    """
    create_map creates a map with the paths between our addresses, from address1 to address2, 
    then from address2 to address3, etc.

    :routes: a list of the directions we gathered from the ORS API
    :coords: a tuple of tuples containing the coordinates of the different addresses
    :return: returns a Folium map with the routes traced between the addresses
    """ 
    # We initialize our map and the DataFrame that will contain the latitude and longitude of every point returned by the API
    m = folium.Map()
    df_route = pd.DataFrame()
        
    # We loop over the routes    
    for route in routes:
        # We access the geometry param of the response we got from the API containing the coordinates of every point
        mls = route.geometry
        points = [(i[1], i[0]) for i in mls]
        
        # We draw the lines based on our list of points
        folium.PolyLine(points, weight=5, opacity=1).add_to(m)
        
        # We populate our DataFrame with the latitude and longitude of every point for each route
        temp = pd.DataFrame(mls).rename(columns={0:'Lon', 1:'Lat'})
        df_route = pd.concat([df_route, temp])
        
    # We create an optimal zoom based on min and max values in our latitude and longitude DataFrame
    sw = df_route[['Lat', 'Lon']].min().values.tolist()
    sw = [sw[0]-0.00005, sw[1]-0.00005]
    ne = df_route[['Lat', 'Lon']].max().values.tolist()
    ne = [ne[0]+0.00005, ne[1]+0.00005]
    m.fit_bounds([sw, ne])
    
    # For every address we defined, we add a marker on the map
    for index, point in enumerate(coords):
        folium.Marker(location=[point[1], point[0]+0.05], 
                      icon=folium.DivIcon(html=f"""<div style="font-family: courier new; color: Green">{addresses[index]}</div>""")
                      ).add_to(m)
    
    return m

In [9]:
# We call our create_map function
m = create_map(routes, coords)

# We can either show the map in the notebook or save to .html format
m
#m.save('./route_map.html')

## Route optimization
The goal of this part will be to optimize the routing we drew on the map to avoid unecessary kilometers and wasting precious time.
\
- Call the API to get the route between each pair of cities
- Calculate the distance, by road, between each pair of cities based on the response from the API
- Optimize the routing based on the distances calculated previously